In [1]:
import pandas as pd
import nltk
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
nltk.download
nltk.download('words')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
import re
import threading
import langdetect as ld
from nltk.corpus import words
import numpy as np

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from multiprocessing import Process
import glob
import shutil
from multiprocessing import Manager
import multiprocessing
import os

In [89]:
df = pd.read_csv('bitcoin_crash_1.csv',on_bad_lines='skip')
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df.head()

,date,text
0,2017-01-01,Will #Bitcoin Crash #Gold? #fintech #cryptocur...
1,2017-01-02,Free 10 Day Bitcoin Crash Course...\n\n&gt;&gt...
2,2017-01-03,RT @BColwell_EMGMKT: Will #Bitcoin Crash #Gold...
3,2017-01-03,RT @BColwell_ECON: Will #Bitcoin Crash #Gold? ...
4,2017-01-03,RT @BColwell_ECON: Will #Bitcoin Crash #Gold? ...


In [90]:
dir_path = "C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Twitter/Bitcoin_Crash/"
file_list = []
for file in os.listdir(dir_path):
    if file.endswith('.csv')and 'bitcoin_crash_1' not in file:
        file_list.append(dir_path + file)
file_list

['C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Twitter/Bitcoin_Crash/bitcoin_crash_2.csv',
 'C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Twitter/Bitcoin_Crash/bitcoin_crash_3.csv',
 'C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Twitter/Bitcoin_Crash/bitcoin_crash_4.csv',
 'C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Twitter/Bitcoin_Crash/bitcoin_crash_5.csv',
 'C:/Users/Admin/Desktop/Columbia/E4524 Capstone/Twitter/Bitcoin_Crash/bitcoin_crash_6.csv']

In [91]:
for file in file_list:
    df2 = pd.read_csv(file,on_bad_lines='skip')
    df2['date'] = pd.to_datetime(df2['date'], errors='coerce')
    df = pd.concat([df,df2]).reset_index(drop = True)
df

,date,text
0,2017-01-01,Will #Bitcoin Crash #Gold? #fintech #cryptocur...
1,2017-01-02,Free 10 Day Bitcoin Crash Course...\n\n&gt;&gt...
2,2017-01-03,RT @BColwell_EMGMKT: Will #Bitcoin Crash #Gold...
3,2017-01-03,RT @BColwell_ECON: Will #Bitcoin Crash #Gold? ...
4,2017-01-03,RT @BColwell_ECON: Will #Bitcoin Crash #Gold? ...
...,...,...
500809,2023-01-18,https://t.co/tYzJ8zmP5c
500810,2023-01-18,WARNING! FINAL BITCOIN CRASH MAY COME BEFORE T...
500811,2023-01-18,BITCOIN CRASH Elon Musk Tanks Market! Money Mo...
500812,2023-01-18,Bitcoin crash or new rally? XRP lawsuit update...


In [92]:
# Remove duplicates
df = df.drop_duplicates(subset=['text']).reset_index(drop = True)
df

,date,text
0,2017-01-01,Will #Bitcoin Crash #Gold? #fintech #cryptocur...
1,2017-01-02,Free 10 Day Bitcoin Crash Course...\n\n&gt;&gt...
2,2017-01-03,RT @BColwell_EMGMKT: Will #Bitcoin Crash #Gold...
3,2017-01-03,RT @BColwell_ECON: Will #Bitcoin Crash #Gold? ...
4,2017-01-03,Will #Bitcoin Crash #Gold? #fintech #cryptocur...
...,...,...
104381,2022-05-06,#Bitcoin CRASH!!! WTF JUST HAPPENED???\n\nhttp...
104382,2022-05-06,@ChartGOD_ @ToddAultIII @Ollie_TradeGod There ...
104383,2022-05-06,"Bitcoin crash spoedupdate: wat zat erachter, e..."
104384,2022-05-06,Cardano price rallied 16% in a single daily ca...


In [94]:
df = df.sort_values(by=['date']).reset_index(drop = True)
df.dropna(inplace = True)
df

,date,text
0,2017-01-01,Will #Bitcoin Crash #Gold? #fintech #cryptocur...
1,2017-01-02,Free 10 Day Bitcoin Crash Course...\n\n&gt;&gt...
2,2017-01-03,RT @BColwell_EMGMKT: Will #Bitcoin Crash #Gold...
3,2017-01-03,RT @BColwell_ECON: Will #Bitcoin Crash #Gold? ...
4,2017-01-03,Will #Bitcoin Crash #Gold? #fintech #cryptocur...
...,...,...
104380,2023-01-18,I have some pretty good pattern reading skills...
104381,2023-01-18,RT @CryptoMichNL: NEW VIDEO\n\n#Bitcoin Crash!...
104382,2023-01-18,@TPostMillennial Bitcoin crash
104383,2023-01-18,RT @CryptoGuider786: #BTC Dump 🔥\n\nIt will Go...


In [95]:
# Remove non-English tweets
counter =0 
tweets = len(df)
def is_english(tweet):
    global counter
    counter = counter + 1    
    if(counter % 10000 == 0):
        print("Completed: "+ str(counter) +"/"+ str(tweets))
    try:
        return ld.detect(tweet) == 'en'
    except:
        return False
df = df[df['text'].apply(is_english)].reset_index(drop = True)
df

Completed: 10000/104385
Completed: 20000/104385
Completed: 30000/104385
Completed: 40000/104385
Completed: 50000/104385
Completed: 60000/104385
Completed: 70000/104385
Completed: 80000/104385
Completed: 90000/104385
Completed: 100000/104385


,date,text
0,2017-01-01,Will #Bitcoin Crash #Gold? #fintech #cryptocur...
1,2017-01-02,Free 10 Day Bitcoin Crash Course...\n\n&gt;&gt...
2,2017-01-03,RT @BColwell_EMGMKT: Will #Bitcoin Crash #Gold...
3,2017-01-03,RT @BColwell_ECON: Will #Bitcoin Crash #Gold? ...
4,2017-01-03,Will #Bitcoin Crash #Gold? #fintech #cryptocur...
...,...,...
96352,2023-01-18,I have some pretty good pattern reading skills...
96353,2023-01-18,RT @CryptoMichNL: NEW VIDEO\n\n#Bitcoin Crash!...
96354,2023-01-18,@TPostMillennial Bitcoin crash
96355,2023-01-18,RT @CryptoGuider786: #BTC Dump 🔥\n\nIt will Go...


In [96]:
df.to_csv('bitcoincrash_duplicate_nonenglish.csv',index = False)

In [5]:
df = pd.read_csv('bitcoincrash_duplicate_nonenglish.csv',usecols = [1,2])
df

,date,text
0,2014-01-01,BITCOIN Start There!! $$!: https://t.co/Bu1539...
1,2014-01-01,RT @CNBCWorld: What's a better bet in 2014: go...
2,2014-01-01,"RT @robertpi: First blog post of 2014, Bitcoin..."
3,2014-01-01,"""Bitcoin Litecoin Survival Guide"" has been pos..."
4,2014-01-01,#bitcoin #btc New Post - Bitcoin Traders Rake ...
...,...,...
2000938,2023-01-18,"Bitcoin inches closer to a 10-year record, as ..."
2000939,2023-01-18,"Current Bitcoin Price:\nUSD $20,692.91\nGBP £1..."
2000940,2023-01-18,"@jimcramer Hey Jim, crypto is one thing, Bitc..."
2000941,2023-01-18,#crypto #bitcoin #altcoin #NFTs #btc #eth #xrp...


In [97]:
# Remove URLs
def remove_URL(sample):
    return re.sub("(?P<url>https?://[^\s]+)", "", sample)

In [98]:
# Lemmatize tweets
lemmatizer = nltk.stem.WordNetLemmatizer()
tokenizer = TweetTokenizer()
detokeniser = TreebankWordDetokenizer()
def lemmatize_text(text):
    return [(lemmatizer.lemmatize(w)) for w in tokenizer.tokenize((text))]
def detokenise_text(tokenised):
    return detokeniser.detokenize(tokenised)

In [99]:
# Remove hashtag if word after hashtag symbol is not in NLTK vocabulary
# otherwise just remove hashtag symbol
word_set = set(words.words())
def get_hashtag_text(word):
    global word_set
    #if not a hahstag return the word
    if(not word.startswith("#")):
        return word    
    #remove hashtag
    word = word[1:]
    return word if (word in word_set) else ""

In [100]:
# Remove punctuation
def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', (word))
        if new_word != '':
            new_words.append(new_word)
    return new_words

In [101]:
counter =0 
tweets = len(df)
def preprocess_tweet(tweet):  
    global counter
    counter = counter + 1    
    if(counter % 10000 == 0):
        print("Completed: "+ str(counter) +"/"+ str(tweets))
    # 1. remove url
    tweet = remove_URL(tweet)
    # 2. lemmatize
    tokenised = lemmatize_text(tweet)
    # 3. keep only hashtags in english dictionary
    tokenised = [get_hashtag_text(word) for word in tokenised if get_hashtag_text(word) != ''] 
    # 4. remove stop words
    stop_words = ["a", "the", "in", "with", "is"]
    tokenised = [word for word in tokenised if word not in stop_words] 
    # 5. replace mentions by 'USER'
    for index, word in enumerate(tokenised):
        if(word.startswith("@")):
            tokenised[index] = "USER"            
    # 6. remove punctuation
    tokenised = remove_punctuation(tokenised)            
    return detokenise_text(tokenised)

In [14]:
## Test
sentence = "RT @robertpi: First blog post of 2014, Bitcoin: A Detailed Yet Easy to Understand Explanation: http://t.co/naTqTPy6Ub"
preprocess_tweet(sentence)

'RT USER First blog post of 2014 Bitcoin A Detailed Yet Easy to Understand Explanation'

In [102]:
df['text'] = df['text'].apply(preprocess_tweet)
df

Completed: 10000/96357
Completed: 20000/96357
Completed: 30000/96357
Completed: 40000/96357
Completed: 50000/96357
Completed: 60000/96357
Completed: 70000/96357
Completed: 80000/96357
Completed: 90000/96357


,date,text
0,2017-01-01,Will Crash Gold
1,2017-01-02,Free 10 Day Bitcoin Crash Course
2,2017-01-03,RT USER Will Crash Gold
3,2017-01-03,RT USER Will Crash Gold
4,2017-01-03,Will Crash Gold
...,...,...
96352,2023-01-18,I have some pretty good pattern reading skill ...
96353,2023-01-18,RT USER NEW VIDEO Crash Whats Happening Watch ...
96354,2023-01-18,USER Bitcoin crash
96355,2023-01-18,RT USER Dump It will Go below 20270


In [103]:
# Remove tweets with a word count of less than 4
def count_words(sentence):
    sentence = str(sentence)
    return len(sentence.split())
df = df[df['text'].map(count_words) > 3].reset_index(drop = True)
df

,date,text
0,2017-01-02,Free 10 Day Bitcoin Crash Course
1,2017-01-03,RT USER Will Crash Gold
2,2017-01-03,RT USER Will Crash Gold
3,2017-01-03,Remember he made Bitcoin crash lie about an ex...
4,2017-01-05,MGTI Why Its Moving gold fiat crash USER
...,...,...
94137,2023-01-18,Why did Bitcoin crash 2022 A disaster for priv...
94138,2023-01-18,I have some pretty good pattern reading skill ...
94139,2023-01-18,RT USER NEW VIDEO Crash Whats Happening Watch ...
94140,2023-01-18,RT USER Dump It will Go below 20270


In [104]:
df.to_csv('bitcoincrash_cleaned.csv',index = False)

In [ ]:
df = pd.read_csv('bitcoincrash_cleaned.csv')

In [105]:
# VADER sentiment analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sentiment_analyser = SentimentIntensityAnalyzer()
df['compound'] = df['text'].apply(lambda x: 'NaN' if pd.isnull(x) else sentiment_analyser.polarity_scores(x)["compound"])
df['positive'] = df['text'].apply(lambda x: 'NaN' if pd.isnull(x) else sentiment_analyser.polarity_scores(x)["pos"])    
df['negative'] = df['text'].apply(lambda x: 'NaN' if pd.isnull(x) else sentiment_analyser.polarity_scores(x)["neg"])    
df['neutral'] = df['text'].apply(lambda x: 'NaN' if pd.isnull(x) else sentiment_analyser.polarity_scores(x)["neu"])
df

,date,text,compound,positive,negative,neutral
0,2017-01-02,Free 10 Day Bitcoin Crash Course,0.1531,0.330,0.270,0.400
1,2017-01-03,RT USER Will Crash Gold,-0.4019,0.000,0.403,0.597
2,2017-01-03,RT USER Will Crash Gold,-0.4019,0.000,0.403,0.597
3,2017-01-03,Remember he made Bitcoin crash lie about an ex...,-0.1280,0.202,0.280,0.518
4,2017-01-05,MGTI Why Its Moving gold fiat crash USER,-0.4019,0.000,0.278,0.722
...,...,...,...,...,...,...
94137,2023-01-18,Why did Bitcoin crash 2022 A disaster for priv...,-0.7783,0.000,0.312,0.688
94138,2023-01-18,I have some pretty good pattern reading skill ...,-0.0516,0.231,0.239,0.530
94139,2023-01-18,RT USER NEW VIDEO Crash Whats Happening Watch ...,-0.4019,0.000,0.231,0.769
94140,2023-01-18,RT USER Dump It will Go below 20270,-0.3818,0.000,0.271,0.729


In [106]:
df.to_csv('bitcoincrash_vader.csv',index = False)

In [12]:
df = pd.read_csv('bitcoincrash_vader.csv')
df

,date,text,compound,positive,negative,neutral
0,2014-01-01,RT USER Whats better bet 2014 gold or bitcoin ...,0.4404,0.244,0.000,0.756
1,2014-01-01,RT USER First blog post of 2014 Bitcoin A Deta...,0.4404,0.172,0.000,0.828
2,2014-01-01,Bitcoin Litecoin Survival Guide ha been posted...,0.3400,0.231,0.000,0.769
3,2014-01-01,New Post Bitcoin Traders Rake Big Profits As B...,0.4404,0.244,0.000,0.756
4,2014-01-01,RT USER Complete list of Bitcoin exchange from...,-0.2960,0.000,0.155,0.845
...,...,...,...,...,...,...
1925981,2023-01-18,Bitcoin inch closer to 10 year record other st...,0.0000,0.000,0.000,1.000
1925982,2023-01-18,Current Bitcoin Price USD 20692 91 GBP 17290 8...,0.0000,0.000,0.000,1.000
1925983,2023-01-18,USER Hey Jim crypto one thing Bitcoin another,0.0000,0.000,0.000,1.000
1925984,2023-01-18,National Australia Bank creates stablecoin cal...,-0.2023,0.095,0.132,0.773


In [107]:
import plotly.express as px
pos = len(df[df['compound'] > 0])
neg = len(df[df['compound'] < 0])
neu = len(df[df['compound'] == 0])
individual = pd.DataFrame(np.zeros((3, 2)), columns = ['sentiment','number'])
individual['sentiment'] = ['positive', 'negative', 'neutral']
individual['number'] = [pos, neg, neu]
individual

,sentiment,number
0,positive,33331
1,negative,57425
2,neutral,3386


In [108]:
fig = px.pie(individual,values='number', names='sentiment', title='Individual Tweets Sentiment',color='sentiment',
             color_discrete_map={'positive':'darkblue',
                                 'negative':'lightsteelblue',
                                 'neutral':'royalblue',},
            width=500, height=450)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [109]:
# daily average sentiment and volume
sentiment = df.groupby(['date']).mean().reset_index(drop = False)
volume = df.groupby(['date']).count().reset_index(drop = False)[['date','text']]
volume.rename(columns={'date': 'date','text': 'volume'},inplace = True)
tweets = sentiment.merge(volume,how = 'inner',on = 'date')
tweets

,date,compound,positive,negative,neutral,volume
0,2017-01-02,0.153100,0.330000,0.270000,0.400000,1
1,2017-01-03,-0.310600,0.067333,0.362000,0.570667,3
2,2017-01-05,-0.188200,0.075042,0.202833,0.722167,24
3,2017-01-06,-0.274089,0.052000,0.324556,0.623444,9
4,2017-01-07,-0.414630,0.000000,0.354700,0.645300,10
...,...,...,...,...,...,...
2064,2023-01-14,-0.470935,0.017750,0.255500,0.726850,20
2065,2023-01-15,-0.485055,0.015250,0.243300,0.741500,20
2066,2023-01-16,-0.254508,0.063846,0.222615,0.713615,13
2067,2023-01-17,-0.351520,0.049200,0.256600,0.694200,10


In [110]:
tweets.to_csv('bitcoincrash_sentiment_and_volume.csv',index = False)

In [24]:
tweets = pd.read_csv('bitcoincrash_sentiment_and_volume.csv')

In [111]:
pos_d = len(tweets[tweets['compound'] > 0])
neg_d = len(tweets[tweets['compound'] < 0])
neu_d = len(tweets[tweets['compound'] == 0])
daily = pd.DataFrame(np.zeros((3, 2)), columns = ['sentiment','number'])
daily['sentiment'] = ['positive', 'negative', 'neutral']
daily['number'] = [pos_d, neg_d, neu_d]
daily

,sentiment,number
0,positive,286
1,negative,1773
2,neutral,10


In [112]:
fig = px.pie(daily,values='number', names='sentiment', title='Daily Twitter Sentiment',color='sentiment',
             color_discrete_map={'positive':'darkblue',
                                 'negative':'lightsteelblue',
                                 'neutral':'royalblue',},
            width=500, height=450)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [113]:
# Plots
import plotly.express as px
fig1 = px.line(tweets, x='date', y=['compound','positive','negative','neutral'],title = 'Sentiment Scores')
fig1.show()
fig2 = px.line(tweets, x='date', y = 'volume', title = 'Volume')
fig2.show()